# Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
from src.read_data import read_file
from src.text_normalization import english_text_normalization
from src.text_processing import data_words, words_to_index_, index_to_word_, word_counts, text_to_sequence, sequences_padding, word_sequence_to_text
import random
from tensorflow.keras.layers import Attention, AdditiveAttention, MultiHeadAttention # type: ignore
from tensorflow.keras.layers import LSTM, GRU, Bidirectional# type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from src.data_split import input_output_split, decoder_input_output
from src.encoder import encoder_without_attention, encoder_with_attention
from src.decoder import decoder_without_attention, decoder_with_attention
from src.encoder_decoder_model import build_model, model_compile, model_fit, save_model, summary
from src.transformers import transformers
import warnings
import os
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Ignore Warnings

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

# Set Early Stop For Encoder-Decoder Model

In [3]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Read Data

In [5]:
Corpus = read_file(file_path='Question Answer Data.txt')
len(Corpus)

3838

In [6]:
random.seed(44)
Corpus = random.sample(Corpus, len(Corpus))

# Split The Data Into Encoder Input And Output

In [7]:
context, question, answer = input_output_split(data=Corpus, text_normalization1=english_text_normalization, text_normalization2=english_text_normalization, text_normalization3 =english_text_normalization)

# Text Processing

**1-Word To Integer (Tokenizer)**

In [8]:
all_words , words = data_words(filter='"#$&*+/:=@[\\]^_{|}~', data=(context + question + answer)) # Set With Unique Words And Find Number Of Unique Words
words_to_index = words_to_index_(words=words) # Convert Word To Integer Index
index_to_words = index_to_word_(words=words)

In [9]:
all_words

10876

**2-Data To Sequence**

In [10]:
context_sequences = text_to_sequence(word_index=words_to_index, data=context) # Convert Text Into Squences Of Integer
question_sequences = text_to_sequence(word_index=words_to_index, data=question) # Convert Text Into Squences Of Integer
answer_sequences = text_to_sequence(word_index=words_to_index, data=answer) # Convert Text Into Squences Of Integer

**3-Max Length**

In [11]:
context_max_length = len(max(context_sequences, key = len)) # Find Max Length
question_max_length = len(max(question_sequences, key = len))
max_length = max(context_max_length, question_max_length)
answer_max_length = len(max(answer_sequences, key=len))# Find Max Length

**4-Encoder Zero Padding**

In [12]:
pad_context = sequences_padding(input_sequence=context_sequences, max_length=max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
pad_question = sequences_padding(input_sequence=question_sequences, max_length=max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)

**5-Generate Decoder Input And Output**

In [13]:
decoder_input, decoder_output = decoder_input_output(output_data= answer_sequences)

**6-Decoder Zero Padding**

In [14]:
pad_decoder_input = sequences_padding(input_sequence=decoder_input, max_length=answer_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
pad_decoder_output = sequences_padding(input_sequence=decoder_output, max_length=answer_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)

**7-Word Occurrence**

In [15]:
word_count = word_counts(data=(context + question + answer)) # Number Of Occurrence Each Word

# Encoder-Decoder Without Attention

In [16]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**1-LSTM**

In [17]:
encoder_model_1 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_1)

decoder_model_1 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_1)

model_1 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_1)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None, 200) │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │  1,392,128 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, None,     │    168,448 │ concatenate[0][0] │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    131,584 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  1,403,004 │ lstm_1[0][0]      │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,182,764 (15.96 MB)

 Trainable params: 4,182,764 (15.96 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model_compile(model = model_1, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_1, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_1,name="LSTM_1")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 191ms/step - loss: 7.7110 - sparse_categorical_accuracy: 0.0434
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 30s 246ms/step - loss: 5.7003 - sparse_categorical_accuracy: 0.0455
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 193ms/step - loss: 5.4950 - sparse_categorical_accuracy: 0.0482
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 20s 166ms/step - loss: 5.3429 - sparse_categorical_accuracy: 0.0493
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 21s 177ms/step - loss: 5.2454 - sparse_categorical_accuracy: 0.0495
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 38s 319ms/step - loss: 5.1660 - sparse_categorical_accuracy: 0.0495
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 28s 224ms/step - loss: 5.0905 - sparse_categorical_accuracy: 0.0495
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 38s 321ms/step - loss: 5.0131 - sparse_categorical_accuracy: 0.0497
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 189ms/step - loss: 4.9330 - sparse_categorical_accuracy: 0.0501
Epoch 10/120
120/12

In [19]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**2-GRU**

In [20]:
encoder_model_2 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_2)

decoder_model_2 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_2)

model_2 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_2)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, None, 200) │          0 │ embedding_2[0][0… │
│ (Concatenate)       │                   │            │ embedding_2[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │  1,392,128 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ [(None, None,     │    126,720 │ concatenate_1[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, None,     │     99,072 │ embedding_3[0][0… │
│                     │ 128), (None,      │            │ gru[0][1]         │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  1,403,004 │ gru_1[0][0]       │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,108,524 (15.67 MB)

 Trainable params: 4,108,524 (15.67 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model_compile(model = model_2, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_2, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_2, name="GRU_1")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 19s 146ms/step - loss: 7.7364 - sparse_categorical_accuracy: 0.0426
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 141ms/step - loss: 5.7934 - sparse_categorical_accuracy: 0.0459
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 145ms/step - loss: 5.5389 - sparse_categorical_accuracy: 0.0487
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 140ms/step - loss: 5.3986 - sparse_categorical_accuracy: 0.0492
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 140ms/step - loss: 5.2777 - sparse_categorical_accuracy: 0.0493
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 18s 148ms/step - loss: 5.1630 - sparse_categorical_accuracy: 0.0495
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 17s 138ms/step - loss: 5.0242 - sparse_categorical_accuracy: 0.0494
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 20s 169ms/step - loss: 4.8953 - sparse_categorical_accuracy: 0.0496
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 21s 174ms/step - loss: 4.7590 - sparse_categorical_accuracy: 0.0512
Epoch 10/120
120/12

In [22]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

**3-Bidirectional**

In [23]:
encoder_model_3 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states = encoder_without_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_3)

decoder_model_3 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_without_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_3)

model_3 =  build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_3)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, None, 200) │          0 │ embedding_4[0][0… │
│ (Concatenate)       │                   │            │ embedding_4[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, None,     │    336,896 │ concatenate_2[0]… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, None, 128) │  1,392,128 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    394,240 │ embedding_5[0][0… │
│                     │ 256), (None,      │            │ concatenate_3[0]… │
│                     │ 256), (None,      │            │ concatenate_4[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None,      │  2,795,132 │ lstm_3[0][0]      │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,005,996 (22.91 MB)

 Trainable params: 6,005,996 (22.91 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model_compile(model = model_3, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_3, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_3,name="Bidirectional_1")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 246ms/step - loss: 7.4406 - sparse_categorical_accuracy: 0.0434
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 37s 307ms/step - loss: 5.6273 - sparse_categorical_accuracy: 0.0465
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 33s 273ms/step - loss: 5.3938 - sparse_categorical_accuracy: 0.0486
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 32s 267ms/step - loss: 5.2318 - sparse_categorical_accuracy: 0.0493
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 34s 284ms/step - loss: 5.1143 - sparse_categorical_accuracy: 0.0494
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 32s 266ms/step - loss: 5.0063 - sparse_categorical_accuracy: 0.0495
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 34s 284ms/step - loss: 4.8886 - sparse_categorical_accuracy: 0.0496
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 35s 295ms/step - loss: 4.7553 - sparse_categorical_accuracy: 0.0496
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 37s 305ms/step - loss: 4.6068 - sparse_categorical_accuracy: 0.0498
Epoch 10/120
120/12

# Encoder-Decoder With Attention

**1-Luong Attension**

In [25]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

*A-LSTM*

In [26]:
encoder_model_4 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_4)

attention_layer = Attention(use_scale=True)
decoder_model_4 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_4, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_4 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_4)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, None, 200) │          0 │ embedding_6[0][0… │
│ (Concatenate)       │                   │            │ embedding_6[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, None, 128) │  1,392,128 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, None,     │    168,448 │ concatenate_5[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, None,     │    131,584 │ embedding_7[0][0… │
│                     │ 128), (None,      │            │ lstm_4[0][1],     │
│                     │ 128), (None,      │            │ lstm_4[0][2]      │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, None, 128) │          1 │ lstm_5[0][0],     │
│ (Attention)         │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, None, 256) │          0 │ lstm_5[0][0],     │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None,      │  2,795,132 │ concatenate_6[0]… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,574,893 (21.27 MB)

 Trainable params: 5,574,893 (21.27 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model_compile(model = model_4, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_4, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_4, name="LSTM_2")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 26s 199ms/step - loss: 3.7715 - sparse_categorical_accuracy: 0.8170
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 198ms/step - loss: 0.9443 - sparse_categorical_accuracy: 0.8892
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 195ms/step - loss: 0.9076 - sparse_categorical_accuracy: 0.8924
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 199ms/step - loss: 0.8749 - sparse_categorical_accuracy: 0.8926
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 195ms/step - loss: 0.8517 - sparse_categorical_accuracy: 0.8931
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 28s 233ms/step - loss: 0.8348 - sparse_categorical_accuracy: 0.8934
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 241ms/step - loss: 0.8212 - sparse_categorical_accuracy: 0.8934
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 26s 213ms/step - loss: 0.8075 - sparse_categorical_accuracy: 0.8937
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 198ms/step - loss: 0.7928 - sparse_categorical_accuracy: 0.8938
Epoch 10/120
120/12

*B-GRU*

In [28]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [29]:
encoder_model_5 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_5)

attention_layer = Attention(use_scale=True)
decoder_model_5 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_5, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_5 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_5)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_8         │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, None, 200) │          0 │ embedding_8[0][0… │
│ (Concatenate)       │                   │            │ embedding_8[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, None, 128) │  1,392,128 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_2 (GRU)         │ [(None, None,     │    126,720 │ concatenate_7[0]… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_3 (GRU)         │ [(None, None,     │     99,072 │ embedding_9[0][0… │
│                     │ 128), (None,      │            │ gru_2[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, None, 128) │          1 │ gru_3[0][0],      │
│ (Attention)         │                   │            │ gru_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, None, 256) │          0 │ gru_3[0][0],      │
│ (Concatenate)       │                   │            │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  2,795,132 │ concatenate_8[0]… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,500,653 (20.98 MB)

 Trainable params: 5,500,653 (20.98 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model_compile(model = model_5, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_5, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_5, name="GRU_2")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 190ms/step - loss: 3.7114 - sparse_categorical_accuracy: 0.8213
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 179ms/step - loss: 0.9399 - sparse_categorical_accuracy: 0.8897
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 180ms/step - loss: 0.8817 - sparse_categorical_accuracy: 0.8922
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 192ms/step - loss: 0.8419 - sparse_categorical_accuracy: 0.8930
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 184ms/step - loss: 0.8096 - sparse_categorical_accuracy: 0.8936
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 181ms/step - loss: 0.7831 - sparse_categorical_accuracy: 0.8942
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 259ms/step - loss: 0.7596 - sparse_categorical_accuracy: 0.8945
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 221ms/step - loss: 0.7399 - sparse_categorical_accuracy: 0.8950
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 203ms/step - loss: 0.7218 - sparse_categorical_accuracy: 0.8956
Epoch 10/120
120/12

*C-Bidirectional*

In [31]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [32]:
encoder_model_6 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_6)

attention_layer = Attention(use_scale=True)
decoder_model_6 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_6, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_6 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_6)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, None, 200) │          0 │ embedding_10[0][… │
│ (Concatenate)       │                   │            │ embedding_10[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ [(None, None,     │    336,896 │ concatenate_9[0]… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, None, 128) │  1,392,128 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 256)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_11      │ (None, 256)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, None,     │    394,240 │ embedding_11[0][… │
│                     │ 256), (None,      │            │ concatenate_10[0… │
│                     │ 256), (None,      │            │ concatenate_11[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, None, 256) │          1 │ lstm_7[0][0],     │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, None, 512) │          0 │ lstm_7[0][0],     │
│ (Concatenate)       │                   │            │ attention_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None,      │  5,579,388 │ concatenate_12[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,790,253 (33.53 MB)

 Trainable params: 8,790,253 (33.53 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model_compile(model = model_6, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_6, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_6, name="Bidirectional_2")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 44s 354ms/step - loss: 3.0896 - sparse_categorical_accuracy: 0.8193
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 58s 471ms/step - loss: 0.9439 - sparse_categorical_accuracy: 0.8908
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 50s 409ms/step - loss: 0.9045 - sparse_categorical_accuracy: 0.8925
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 48s 403ms/step - loss: 0.8573 - sparse_categorical_accuracy: 0.8930
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 54s 452ms/step - loss: 0.8185 - sparse_categorical_accuracy: 0.8934
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 51s 429ms/step - loss: 0.7968 - sparse_categorical_accuracy: 0.8934
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 48s 403ms/step - loss: 0.7809 - sparse_categorical_accuracy: 0.8936
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 46s 382ms/step - loss: 0.7708 - sparse_categorical_accuracy: 0.8939
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 47s 394ms/step - loss: 0.7509 - sparse_categorical_accuracy: 0.8947
Epoch 10/120
120/12

**2-Bahdanau Attension**

*A-LSTM*

In [34]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [35]:
encoder_model_7 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_7)

attention_layer = AdditiveAttention(use_scale=True)
decoder_model_7 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_7, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_7 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_7)

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, None, 200) │          0 │ embedding_12[0][… │
│ (Concatenate)       │                   │            │ embedding_12[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, None, 128) │  1,392,128 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_8 (LSTM)       │ [(None, None,     │    168,448 │ concatenate_13[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ [(None, None,     │    131,584 │ embedding_13[0][… │
│                     │ 128), (None,      │            │ lstm_8[0][1],     │
│                     │ 128), (None,      │            │ lstm_8[0][2]      │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention  │ (None, None, 128) │        128 │ lstm_9[0][0],     │
│ (AdditiveAttention) │                   │            │ lstm_8[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_14      │ (None, None, 256) │          0 │ lstm_9[0][0],     │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, None,      │  2,795,132 │ concatenate_14[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,575,020 (21.27 MB)

 Trainable params: 5,575,020 (21.27 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model_compile(model = model_7, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_7, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_7, name="LSTM_3")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 35s 276ms/step - loss: 3.8254 - sparse_categorical_accuracy: 0.8149
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 203ms/step - loss: 0.9530 - sparse_categorical_accuracy: 0.8891
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 207ms/step - loss: 0.9181 - sparse_categorical_accuracy: 0.8904
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 200ms/step - loss: 0.8780 - sparse_categorical_accuracy: 0.8925
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 199ms/step - loss: 0.8426 - sparse_categorical_accuracy: 0.8932
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 210ms/step - loss: 0.8199 - sparse_categorical_accuracy: 0.8933
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 35s 289ms/step - loss: 0.7977 - sparse_categorical_accuracy: 0.8932
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 259ms/step - loss: 0.7778 - sparse_categorical_accuracy: 0.8933
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 210ms/step - loss: 0.7611 - sparse_categorical_accuracy: 0.8934
Epoch 10/120
120/12

*B-GRU*

In [37]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [38]:
encoder_model_8 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_8)

attention_layer = Attention(use_scale=True)
decoder_model_8 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_8, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_8 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_8)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_14        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_15      │ (None, None, 200) │          0 │ embedding_14[0][… │
│ (Concatenate)       │                   │            │ embedding_14[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, None, 128) │  1,392,128 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_4 (GRU)         │ [(None, None,     │    126,720 │ concatenate_15[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_5 (GRU)         │ [(None, None,     │     99,072 │ embedding_15[0][… │
│                     │ 128), (None,      │            │ gru_4[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_3         │ (None, None, 128) │          1 │ gru_5[0][0],      │
│ (Attention)         │                   │            │ gru_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_16      │ (None, None, 256) │          0 │ gru_5[0][0],      │
│ (Concatenate)       │                   │            │ attention_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, None,      │  2,795,132 │ concatenate_16[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,500,653 (20.98 MB)

 Trainable params: 5,500,653 (20.98 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model_compile(model = model_8, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_8, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_8, name="GRU_3")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 228ms/step - loss: 3.6251 - sparse_categorical_accuracy: 0.8205
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 226ms/step - loss: 0.9429 - sparse_categorical_accuracy: 0.8895
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 179ms/step - loss: 0.8934 - sparse_categorical_accuracy: 0.8923
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 180ms/step - loss: 0.8464 - sparse_categorical_accuracy: 0.8927
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 226ms/step - loss: 0.8121 - sparse_categorical_accuracy: 0.8935
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 224ms/step - loss: 0.7821 - sparse_categorical_accuracy: 0.8939
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 212ms/step - loss: 0.7569 - sparse_categorical_accuracy: 0.8942
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 182ms/step - loss: 0.7335 - sparse_categorical_accuracy: 0.8947
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 180ms/step - loss: 0.7151 - sparse_categorical_accuracy: 0.8955
Epoch 10/120
120/12

*C-Bidirectional*

In [40]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [41]:
encoder_model_9 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_9)

attention_layer = AdditiveAttention(use_scale=True)
decoder_model_9 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_9, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_9 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_9)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_17      │ (None, None, 200) │          0 │ embedding_16[0][… │
│ (Concatenate)       │                   │            │ embedding_16[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ [(None, None,     │    336,896 │ concatenate_17[0… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_17        │ (None, None, 128) │  1,392,128 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 256)       │          0 │ bidirectional_2[… │
│ (Concatenate)       │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_19      │ (None, 256)       │          0 │ bidirectional_2[… │
│ (Concatenate)       │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, None,     │    394,240 │ embedding_17[0][… │
│                     │ 256), (None,      │            │ concatenate_18[0… │
│                     │ 256), (None,      │            │ concatenate_19[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention… │ (None, None, 256) │        256 │ lstm_11[0][0],    │
│ (AdditiveAttention) │                   │            │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, None, 512) │          0 │ lstm_11[0][0],    │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, None,      │  5,579,388 │ concatenate_20[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,790,508 (33.53 MB)

 Trainable params: 8,790,508 (33.53 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
model_compile(model = model_9, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_9, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_9, name="Bidirectional_3")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 58s 462ms/step - loss: 3.0767 - sparse_categorical_accuracy: 0.8190
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 52s 431ms/step - loss: 0.9419 - sparse_categorical_accuracy: 0.8904
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 52s 428ms/step - loss: 0.8960 - sparse_categorical_accuracy: 0.8925
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 64s 533ms/step - loss: 0.8484 - sparse_categorical_accuracy: 0.8932
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 53s 440ms/step - loss: 0.8047 - sparse_categorical_accuracy: 0.8936
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 53s 439ms/step - loss: 0.7739 - sparse_categorical_accuracy: 0.8939
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 54s 448ms/step - loss: 0.7448 - sparse_categorical_accuracy: 0.8937
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 61s 513ms/step - loss: 0.7148 - sparse_categorical_accuracy: 0.8940
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 57s 473ms/step - loss: 0.6894 - sparse_categorical_accuracy: 0.8947
Epoch 10/120
120/12

**3-MultiHead Attesnion**

*A-LSTM*

In [43]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [44]:
encoder_model_10 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_10)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_10 = LSTM(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model= decoder_model_10, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_10 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_10)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_18        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, None, 200) │          0 │ embedding_18[0][… │
│ (Concatenate)       │                   │            │ embedding_18[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_19        │ (None, None, 128) │  1,392,128 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_12 (LSTM)      │ [(None, None,     │    168,448 │ concatenate_21[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_13 (LSTM)      │ [(None, None,     │    131,584 │ embedding_19[0][… │
│                     │ 128), (None,      │            │ lstm_12[0][1],    │
│                     │ 128), (None,      │            │ lstm_12[0][2]     │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │     66,048 │ lstm_12[0][0],    │
│ (MultiHeadAttentio… │ 128), (None, 4,   │            │ lstm_13[0][0],    │
│                     │ None, None)]      │            │ lstm_12[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_22      │ (None, None, 256) │          0 │ lstm_13[0][0],    │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, None,      │  2,795,132 │ concatenate_22[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,640,940 (21.52 MB)

 Trainable params: 5,640,940 (21.52 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model_compile(model = model_10, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_10, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_10, name="LSTM_4")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 189ms/step - loss: 3.5829 - sparse_categorical_accuracy: 0.8153
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 191ms/step - loss: 0.9177 - sparse_categorical_accuracy: 0.8896
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 30s 249ms/step - loss: 0.8683 - sparse_categorical_accuracy: 0.8926
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 226ms/step - loss: 0.8385 - sparse_categorical_accuracy: 0.8930
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 209ms/step - loss: 0.8251 - sparse_categorical_accuracy: 0.8934
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 193ms/step - loss: 0.8036 - sparse_categorical_accuracy: 0.8935
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 227ms/step - loss: 0.7841 - sparse_categorical_accuracy: 0.8935
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 27s 229ms/step - loss: 0.7705 - sparse_categorical_accuracy: 0.8935
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 28s 236ms/step - loss: 0.7582 - sparse_categorical_accuracy: 0.8936
Epoch 10/120
120/12

*B-GRU*

In [46]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [47]:
encoder_model_11= GRU(units=128, return_sequences=True, return_state=True, seed=33)
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_11)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_11 = GRU(units=128, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_11, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_11 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_11)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_20        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_10      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_23      │ (None, None, 200) │          0 │ embedding_20[0][… │
│ (Concatenate)       │                   │            │ embedding_20[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_21        │ (None, None, 128) │  1,392,128 │ input_layer_10[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_6 (GRU)         │ [(None, None,     │    126,720 │ concatenate_23[0… │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_7 (GRU)         │ [(None, None,     │     99,072 │ embedding_21[0][… │
│                     │ 128), (None,      │            │ gru_6[0][1]       │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │     66,048 │ gru_6[0][0],      │
│ (MultiHeadAttentio… │ 128), (None, 4,   │            │ gru_7[0][0],      │
│                     │ None, None)]      │            │ gru_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_24      │ (None, None, 256) │          0 │ gru_7[0][0],      │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, None,      │  2,795,132 │ concatenate_24[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,566,700 (21.24 MB)

 Trainable params: 5,566,700 (21.24 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model_compile(model = model_11, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_11, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_11, name="GRU_4")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 32s 240ms/step - loss: 3.4416 - sparse_categorical_accuracy: 0.8290
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 32s 270ms/step - loss: 0.9085 - sparse_categorical_accuracy: 0.8922
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 210ms/step - loss: 0.8529 - sparse_categorical_accuracy: 0.8922
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 201ms/step - loss: 0.8257 - sparse_categorical_accuracy: 0.8925
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 191ms/step - loss: 0.8097 - sparse_categorical_accuracy: 0.8931
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - loss: 0.7859 - sparse_categorical_accuracy: 0.8933
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 242ms/step - loss: 0.7660 - sparse_categorical_accuracy: 0.8933
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 257ms/step - loss: 0.7467 - sparse_categorical_accuracy: 0.8935
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 26s 217ms/step - loss: 0.7328 - sparse_categorical_accuracy: 0.8938
Epoch 10/120
120/12

*C-Bidirectional*

In [49]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

In [50]:
encoder_model_12 = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, seed=33))
context_inputs, question_inputs, encoder_states, encoder_outputs = encoder_with_attention(input_dim_encoder = all_words, output_dim_encoder = 100, encoder_model= encoder_model_12)

attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
decoder_model_12 = LSTM(units=256, return_sequences=True, return_state=True, seed=33)
decoder_inputs, decoder_outputs  = decoder_with_attention(encoder_states = encoder_states, input_dim_decoder = all_words, output_dim_decoder = 128, unit = all_words,  decoder_model=decoder_model_12, attention_layer=attention_layer, encoder_outputs=encoder_outputs)

model_12 = build_model(context_input= context_inputs, question_input= question_inputs, decoder_inputs = decoder_inputs, decoder_output= decoder_outputs)
summary(model=model_12)

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question            │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, None, 100) │  1,087,600 │ context[0][0],    │
│ (Embedding)         │                   │            │ question[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_25      │ (None, None, 200) │          0 │ embedding_22[0][… │
│ (Concatenate)       │                   │            │ embedding_22[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ [(None, None,     │    336,896 │ concatenate_25[0… │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, None, 128) │  1,392,128 │ input_layer_11[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_26      │ (None, 256)       │          0 │ bidirectional_3[… │
│ (Concatenate)       │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 256)       │          0 │ bidirectional_3[… │
│ (Concatenate)       │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_15 (LSTM)      │ [(None, None,     │    394,240 │ embedding_23[0][… │
│                     │ 256), (None,      │            │ concatenate_26[0… │
│                     │ 256), (None,      │            │ concatenate_27[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ [(None, None,     │    131,712 │ bidirectional_3[… │
│ (MultiHeadAttentio… │ 256), (None, 4,   │            │ lstm_15[0][0],    │
│                     │ None, None)]      │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, None, 512) │          0 │ lstm_15[0][0],    │
│ (Concatenate)       │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, None,      │  5,579,388 │ concatenate_28[0… │
│                     │ 10876)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,921,964 (34.03 MB)

 Trainable params: 8,921,964 (34.03 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model_compile(model = model_12, optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
history = model_fit(model = model_12, epochs = 120, batch_size = 32, context_data =  pad_context, question_data=pad_question, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output, early_stop = early_stopping)
save_model(model=model_12, name="Bidirectional_4")

Epoch 1/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 52s 413ms/step - loss: 3.0288 - sparse_categorical_accuracy: 0.8166
Epoch 2/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 49s 407ms/step - loss: 0.9073 - sparse_categorical_accuracy: 0.8913
Epoch 3/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 49s 408ms/step - loss: 0.8500 - sparse_categorical_accuracy: 0.8924
Epoch 4/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 52s 437ms/step - loss: 0.8077 - sparse_categorical_accuracy: 0.8929
Epoch 5/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 47s 391ms/step - loss: 0.7843 - sparse_categorical_accuracy: 0.8932
Epoch 6/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 54s 451ms/step - loss: 0.7592 - sparse_categorical_accuracy: 0.8934
Epoch 7/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 48s 401ms/step - loss: 0.7398 - sparse_categorical_accuracy: 0.8935
Epoch 8/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 47s 391ms/step - loss: 0.7109 - sparse_categorical_accuracy: 0.8935
Epoch 9/120
120/120 ━━━━━━━━━━━━━━━━━━━━ 55s 460ms/step - loss: 0.6884 - sparse_categorical_accuracy: 0.8938
Epoch 10/120
120/12

# Transformers

In [1]:
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
context = 'new york city has over 28,000 acres (110 km2) of municipal parkland and 14 miles (23 km) of public beaches. parks in new york city include central park, prospect park, flushing meadows–corona park, forest park, and washington square park. the largest municipal park in the city is pelham bay park with 2,700 acres (1,093 ha)'
question = 'new york has approximately how many acres of parks?'
transformers(model_name=model_name, context=context, question=question)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.



Context: new york city has over 28,000 acres (110 km2) of municipal parkland and 14 miles (23 km) of public beaches. parks in new york city include central park, prospect park, flushing meadows–corona park, forest park, and washington square park. the largest municipal park in the city is pelham bay park with 2,700 acres (1,093 ha)
Question: new york has approximately how many acres of parks?
Answer: 28 , 000
